In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/current_customers-1(Powered by MaxAI).csv')

In [ ]:
df.head()

,acct_id,extenders,wireless_clients_count,wired_clients_count,rx_avg_bps,tx_avg_bps,rx_p95_bps,tx_p95_bps,rx_max_bps,tx_max_bps,...,city,state,whole_home_wifi,wifi_security,wifi_security_plus,premium_tech_pro,identity_protection,family_identity_protection,total_shield,youtube_tv
0,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,0,3,2,1.849822e+06,2.229501e+06,2.652856e+06,2.243758e+06,6.606018e+06,2.390025e+06,...,ROMA,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00002ec815678c6b9837ad4c9db82ac7fe5fb6af5dc229...,1,13,6,5.316781e+05,3.170011e+05,1.159275e+06,3.309592e+05,1.947402e+06,3.774517e+05,...,CARROLLTON,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000050825287b713415614a237757da4cd7517365cbce2...,1,9,3,2.657035e+05,9.632467e+03,1.250417e+06,3.180222e+04,1.880263e+06,6.833266e+04,...,AZLE,TX,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,000060be4aa292815abc44ab6fe96015b89e83b21c8a63...,0,3,3,1.077448e+06,4.149613e+05,1.576695e+06,4.234751e+05,1.684422e+06,4.245802e+05,...,VALPARAISO,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00008c39885815e42a0bb750cee199cd4da741a5645705...,1,19,7,6.083613e+05,4.780259e+05,1.349913e+06,6.553424e+05,2.472542e+06,1.111929e+06,...,STRATFORD,CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.isnull().sum()

,0
acct_id,0
extenders,0
wireless_clients_count,0
wired_clients_count,0
rx_avg_bps,0
tx_avg_bps,0
rx_p95_bps,0
tx_p95_bps,0
rx_max_bps,0
tx_max_bps,0


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import requests
from typing import List, Dict
from tqdm import tqdm
import time

def test_scenarios():
    # Initialize system
    system = ISPRecommendationSystem(
        csv_path="current_customers-1(Powered by MaxAI).csv",
        samba_nova_key=""
    )

    # Modify prompt to include product catalog and request specific recommendations
    system_prompt = """Using ONLY the following product catalog, analyze the scenario and recommend the most suitable products.
    Do not recommend any products or features not listed in this catalog.

    Available Products:
    1. Internet Plans:
    - Fiber 500: 500Mbps, standard WIFI router, $45/mo
    - Fiber 1 Gig: 1Gbps, standard WIFI router, $65/mo
    - Fiber 2 Gig: 2Gbps, upgraded WIFI router + extender, $99/mo
    - Fiber 5 Gig: 5Gbps, premium router, $129/mo
    - Fiber 7 Gig: 7Gbps, premium router + free extender, $299/mo

    2. Add-on Services:
    - Additional Extender: $5/mo per extender
    - Whole-Home Wi-Fi: Up to 2 additional extenders (for ≤2 Gig), 1 extender (for 5/7 Gig), $10/mo
    - Unbreakable Wi-Fi: Backup internet, 130GB 4G LTE data, $25/mo
    - Wi-Fi Security: Advanced security, parental controls, $5/mo
    - Wi-Fi Security Plus: Security + Multi-Device + VPN + Password Manager, $10/mo
    - Total Shield: Security for 10 devices + VPN + Password Manager, $10/mo
    - My Premium Tech Pro: $10/mo
    - Identity Protection: Personal information monitoring, $10/mo

    Return your response in JSON format with the following structure:
    {
        "recommendations": {
            "internet_plan": {
                "name": "exact plan name from catalog",
                "price": number,
                "reasoning": "string"
            },
            "recommended_addons": [
                {
                    "name": "exact addon name from catalog",
                    "price": number,
                    "reasoning": "string"
                }
            ],
            "total_monthly_cost": number,
            "notes": ["list", "of", "important", "considerations"]
        }
    }

    Scenario:"""

    scenarios = [
        # Budget-conscious scenarios
        f"""{system_prompt}
        Single student living in a studio apartment:
        - Limited budget (maximum $50/month)
        - Uses internet for online classes and streaming
        - Has a laptop and smartphone
        - Basic security needs
        """,

        # Family scenarios
        f"""{system_prompt}
        Large family of 6:
        - Parents working from home
        - 4 kids doing online schooling
        - Multiple smart TVs and gaming consoles
        - Security and parental controls are crucial
        - Large 3-bedroom house with basement
        - Budget up to $150/month
        """,

        # Professional scenarios
        f"""{system_prompt}
        Software developer working remotely:
        - Needs very high-speed internet
        - Large file uploads/downloads
        - Multiple development servers
        - Video conferencing
        - Lives in a high-rise apartment
        - Budget isn't a constraint
        """
    ]

    print("Starting scenario testing...\n")

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'='*80}")
        print(f"Scenario {i}:")
        print(scenario.split("Scenario:")[-1].strip())
        print("\nAnalyzing scenario...")

        try:
            response = system.analyze_scenario(scenario)
            print("\nRecommendations (JSON):")
            print(response)
        except Exception as e:
            print(f"Error processing scenario {i}: {str(e)}")

        print(f"{'='*80}")

        if i < len(scenarios):
            print("\nProcessing next scenario in 2 seconds...")
            time.sleep(2)

def main():
    print("Testing ISP Recommendation System...")
    test_scenarios()
    print("\nTesting completed!")

if __name__ == "__main__":
    main()

Testing ISP Recommendation System...


<ipython-input-2-7d790c3d2d70>:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Loading CSV data...
Processing 1000 customer records...


100%|██████████| 1000/1000 [00:07<00:00, 128.08it/s]


Created embeddings for 1000 customers
Starting scenario testing...


Scenario 1:
Single student living in a studio apartment:
        - Limited budget (maximum $50/month)
        - Uses internet for online classes and streaming
        - Has a laptop and smartphone
        - Basic security needs

Analyzing scenario...

Recommendations (JSON):
```
{
    "recommendations": {
        "internet_plan": {
            "name": "Fiber 500",
            "price": 45,
            "reasoning": "The customer's limited budget and basic internet needs for online classes and streaming make the Fiber 500 plan the most suitable option. It provides sufficient speed for their requirements and fits within their budget."
        },
        "recommended_addons": [
            {
                "name": "Wi-Fi Security",
                "price": 5,
                "reasoning": "Given the customer's basic security needs, the Wi-Fi Security add-on provides advanced security and parental controls at an affordable 

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import requests
from typing import List, Dict
from tqdm import tqdm
import time

def test_scenarios():
    # Initialize system
    system = ISPRecommendationSystem(
        csv_path="current_customers-1(Powered by MaxAI).csv",
        samba_nova_key=""
    )

    # Base prompt with product catalog and instruction for JSON output
    base_prompt = """Based ONLY on this product catalog, recommend the most suitable products:

    Available Products:
    1. Internet Plans:
    - Fiber 500: 500Mbps, standard WIFI router, $45/mo
    - Fiber 1 Gig: 1Gbps, standard WIFI router, $65/mo
    - Fiber 2 Gig: 2Gbps, upgraded WIFI router + extender, $99/mo
    - Fiber 5 Gig: 5Gbps, premium router, $129/mo
    - Fiber 7 Gig: 7Gbps, premium router + free extender, $299/mo

    2. Add-on Services:
    - Additional Extender: $5/mo per extender
    - Whole-Home Wi-Fi: Up to 2 additional extenders (for ≤2 Gig), 1 extender (for 5/7 Gig), $10/mo
    - Unbreakable Wi-Fi: Backup internet, 130GB 4G LTE data, $25/mo
    - Wi-Fi Security: Advanced security, parental controls, $5/mo
    - Wi-Fi Security Plus: Security + Multi-Device + VPN + Password Manager, $10/mo
    - Total Shield: Security for 10 devices + VPN + Password Manager, $10/mo
    - My Premium Tech Pro: $10/mo
    - Identity Protection: Personal information monitoring, $10/mo

    Provide recommendations in this JSON format:
    {
        "recommendations": {
            "internet_plan": {
                "name": "exact plan name",
                "price": number,
                "reasoning": "string"
            },
            "recommended_addons": [
                {
                    "name": "exact addon name",
                    "price": number,
                    "reasoning": "string"
                }
            ],
            "total_monthly_cost": number,
            "notes": ["list", "of", "important", "considerations"]
        }
    }

    Customer scenario:"""

    scenarios = [
        # Budget-conscious scenarios
        base_prompt + """
        Single student living in a studio apartment:
        - Limited budget (maximum $50/month)
        - Uses internet for online classes and streaming
        - Has a laptop and smartphone
        - Basic security needs
        """,

        # Family scenarios
        base_prompt + """
        Large family of 6:
        - Parents working from home
        - 4 kids doing online schooling
        - Multiple smart TVs and gaming consoles
        - Security and parental controls are crucial
        - Large 3-bedroom house with basement
        - Budget up to $150/month
        """,

        # Professional scenarios
        base_prompt + """
        Software developer working remotely:
        - Needs very high-speed internet
        - Large file uploads/downloads
        - Multiple development servers
        - Video conferencing
        - Lives in a high-rise apartment
        - Budget isn't a constraint
        """
    ]

    print("Starting scenario testing...\n")

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'='*80}")
        print(f"Scenario {i}:")
        print(scenario.split("Customer scenario:")[-1].strip())
        print("\nAnalyzing scenario...")

        try:
            response = system.analyze_scenario(scenario)
            print("\nRecommendations (JSON):")
            print(response)
        except Exception as e:
            print(f"Error processing scenario {i}: {str(e)}")

        print(f"{'='*80}")

        if i < len(scenarios):
            print("\nProcessing next scenario in 2 seconds...")
            time.sleep(2)

def main():
    print("Testing ISP Recommendation System...")
    test_scenarios()
    print("\nTesting completed!")

if __name__ == "__main__":
    main()

Testing ISP Recommendation System...


<ipython-input-2-7d790c3d2d70>:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Loading CSV data...
Processing 1000 customer records...


100%|██████████| 1000/1000 [00:06<00:00, 146.29it/s]


Created embeddings for 1000 customers
Starting scenario testing...


Scenario 1:
Single student living in a studio apartment:
        - Limited budget (maximum $50/month)
        - Uses internet for online classes and streaming
        - Has a laptop and smartphone
        - Basic security needs

Analyzing scenario...

Recommendations (JSON):
Based on the customer scenario, here is a recommendation in the requested JSON format:

```
{
    "recommendations": {
        "internet_plan": {
            "name": "Fiber 500",
            "price": 45,
            "reasoning": "Meets the customer's limited budget of $50/month and provides sufficient speed for online classes and streaming with a laptop and smartphone."
        },
        "recommended_addons": [
            {
                "name": "Wi-Fi Security",
                "price": 5,
                "reasoning": "Provides basic security features, including advanced security and parental controls, which meets the customer's basic securit

In [ ]:
class ScenarioGenerator:
    def __init__(self):
        # Define components for scenario building
        self.usage_profiles = {
            "streaming": {
                "basic": "Occasional Netflix and YouTube",
                "moderate": "Daily streaming on multiple devices",
                "heavy": "4K streaming and content creation",
                "professional": "Multiple 4K streams and content production"
            },
            "gaming": {
                "casual": "Occasional online gaming",
                "regular": "Daily gaming sessions",
                "competitive": "Competitive gaming and streaming",
                "professional": "Esports and professional streaming"
            },
            "work": {
                "basic": "Email and web browsing",
                "moderate": "Regular video calls",
                "heavy": "Multiple video conferences",
                "professional": "Development and large file transfers"
            }
        }

        self.device_profiles = {
            "minimal": ["Smartphone", "Laptop"],
            "standard": ["Smartphones", "Laptops", "Smart TV"],
            "advanced": ["Multiple smartphones", "Gaming PC", "Smart TVs", "Tablets"],
            "professional": ["Workstations", "Servers", "Multiple monitors", "Professional equipment"]
        }

        self.location_types = {
            "small": "Studio apartment",
            "medium": "2-bedroom apartment",
            "large": "3-bedroom house",
            "complex": "Multi-story house with basement"
        }

        self.budget_ranges = {
            "budget": "Under $50/month",
            "standard": "$50-100/month",
            "premium": "$100-200/month",
            "unlimited": "No budget constraints"
        }

    def generate_scenario(self,
                         usage_type: str,
                         intensity: str,
                         devices: str,
                         location: str,
                         budget: str,
                         additional_reqs: List[str] = None) -> str:
        """Generate a detailed scenario with product catalog context"""
        product_catalog = """
        Available Products:
        1. Internet Plans:
        - Fiber 500: 500Mbps, standard WIFI router, $45/mo
        - Fiber 1 Gig: 1Gbps, standard WIFI router, $65/mo
        - Fiber 2 Gig: 2Gbps, upgraded WIFI router + extender, $99/mo
        - Fiber 5 Gig: 5Gbps, premium router, $129/mo
        - Fiber 7 Gig: 7Gbps, premium router + free extender, $299/mo

        2. Add-on Services:
        - Additional Extender: $5/mo per extender
        - Whole-Home Wi-Fi: Up to 2 additional extenders (for ≤2 Gig), 1 extender (for 5/7 Gig), $10/mo
        - Unbreakable Wi-Fi: Backup internet, 130GB 4G LTE data, $25/mo
        - Wi-Fi Security: Advanced security, parental controls, $5/mo
        - Wi-Fi Security Plus: Security + Multi-Device + VPN + Password Manager, $10/mo
        - Total Shield: Security for 10 devices + VPN + Password Manager, $10/mo
        - My Premium Tech Pro: $10/mo
        - Identity Protection: Personal information monitoring, $10/mo

        Based on the following customer profile, recommend products from the catalog above in JSON format:
        {
            "recommendations": {
                "internet_plan": {
                    "name": "exact plan name",
                    "price": number,
                    "reasoning": "string"
                },
                "recommended_addons": [
                    {
                        "name": "exact addon name",
                        "price": number,
                        "reasoning": "string"
                    }
                ],
                "total_monthly_cost": number,
                "notes": ["list", "of", "important", "considerations"]
            }
        }

        Customer Profile:"""

        scenario = f"""
        {product_catalog}

        Customer Details:
        - Usage: {self.usage_profiles[usage_type][intensity]}
        - Devices: {', '.join(self.device_profiles[devices])}
        - Location: {self.location_types[location]}
        - Budget: {self.budget_ranges[budget]}
        """

        if additional_reqs:
            scenario += "\nAdditional Requirements:\n"
            scenario += "\n".join(f"- {req}" for req in additional_reqs)

        return scenario.strip()

def test_comprehensive_scenarios():
    # Initialize system and scenario generator
    system = ISPRecommendationSystem(
        csv_path="current_customers-1(Powered by MaxAI).csv",
        samba_nova_key=""
    )
    generator = ScenarioGenerator()

    # Define test scenarios with same scenarios as before...
    scenarios = [
        # Ultra-high performance scenario
        generator.generate_scenario(
            usage_type="gaming",
            intensity="professional",
            devices="professional",
            location="large",
            budget="unlimited",
            additional_reqs=[
                "Need lowest possible latency",
                "24/7 uptime requirement",
                "Multiple simultaneous 4K streams",
                "Professional tech support",
                "Backup internet connection"
            ]
        ),
        # ... (rest of the scenarios remain the same)
    ]

    print("Starting comprehensive scenario testing...\n")

    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'='*80}")
        print(f"Scenario {i}:")
        print(scenario.split("Customer Details:")[-1].strip())  # Only print customer details
        print("\nAnalyzing scenario...")

        try:
            response = system.analyze_scenario(scenario)
            print("\nRecommendations (JSON):")
            print(response)
        except Exception as e:
            print(f"Error processing scenario {i}: {str(e)}")
            # Add retry logic
            if "503" in str(e):
                print("Service temporarily unavailable. Retrying in 5 seconds...")
                time.sleep(5)
                try:
                    response = system.analyze_scenario(scenario)
                    print("\nRecommendations (JSON) after retry:")
                    print(response)
                except Exception as retry_e:
                    print(f"Retry failed: {str(retry_e)}")

        print(f"{'='*80}")

        if i < len(scenarios):
            print("\nProcessing next scenario in 2 seconds...")
            time.sleep(2)

def main():
    print("Testing Comprehensive ISP Scenarios...")
    test_comprehensive_scenarios()
    print("\nTesting completed!")

if __name__ == "__main__":
    main()

Testing Comprehensive ISP Scenarios...


<ipython-input-2-7d790c3d2d70>:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Loading CSV data...
Processing 1000 customer records...


100%|██████████| 1000/1000 [00:07<00:00, 130.90it/s]


Created embeddings for 1000 customers
Starting comprehensive scenario testing...


Scenario 1:
- Usage: Esports and professional streaming
        - Devices: Workstations, Servers, Multiple monitors, Professional equipment
        - Location: 3-bedroom house
        - Budget: No budget constraints
        
Additional Requirements:
- Need lowest possible latency
- 24/7 uptime requirement
- Multiple simultaneous 4K streams
- Professional tech support
- Backup internet connection

Analyzing scenario...

Recommendations (JSON):
Based on the customer profile, I recommend the following internet plan and add-on services:

```
{
  "recommendations": {
    "internet_plan": {
      "name": "Fiber 5 Gig",
      "price": 129,
      "reasoning": "The customer requires low latency, 24/7 uptime, and multiple simultaneous 4K streams for esports and professional streaming. The Fiber 5 Gig plan provides the necessary speed and reliability to support these demands."
    },
    "recommended_addons": [
   

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import requests
from typing import List, Dict
from tqdm import tqdm
import time

def test_customer_scenarios():
    # Initialize system
    system = ISPRecommendationSystem(
        csv_path="current_customers-1(Powered by MaxAI).csv",
        samba_nova_key=""
    )

    # Base prompt that includes product catalog but allows flexible customer queries
    base_prompt = """You are an ISP sales assistant. Using ONLY the following product catalog, recommend the most suitable products for the customer's needs.

    Available Products:
    1. Internet Plans:
    - Fiber 500: 500Mbps, standard WIFI router, $45/mo
    - Fiber 1 Gig: 1Gbps, standard WIFI router, $65/mo
    - Fiber 2 Gig: 2Gbps, upgraded WIFI router + extender, $99/mo
    - Fiber 5 Gig: 5Gbps, premium router, $129/mo
    - Fiber 7 Gig: 7Gbps, premium router + free extender, $299/mo

    2. Add-on Services:
    - Additional Extender: $5/mo per extender
    - Whole-Home Wi-Fi: Up to 2 additional extenders (for ≤2 Gig), 1 extender (for 5/7 Gig), $10/mo
    - Unbreakable Wi-Fi: Backup internet, 130GB 4G LTE data, $25/mo
    - Wi-Fi Security: Advanced security, parental controls, $5/mo
    - Wi-Fi Security Plus: Security + Multi-Device + VPN + Password Manager, $10/mo
    - Total Shield: Security for 10 devices + VPN + Password Manager, $10/mo
    - My Premium Tech Pro: $10/mo
    - Identity Protection: Personal information monitoring, $10/mo

    Provide recommendations in JSON format:
    {
        "recommendations": {
            "internet_plan": {
                "name": "exact plan name",
                "price": number,
                "reasoning": "string"
            },
            "recommended_addons": [
                {
                    "name": "exact addon name",
                    "price": number,
                    "reasoning": "string"
                }
            ],
            "total_monthly_cost": number,
            "notes": ["list", "of", "important", "considerations"]
        }
    }

    Customer Query:"""

    # Example of how judges can test with various query formats
    sample_queries = [
        # Natural conversation style
        """Hi, I'm looking for internet for my new apartment. I live alone and mainly use it for Netflix
        and some work from home. Don't need anything too fancy, trying to keep it under $50 if possible.""",

        # List style
        """Need internet for:
        - Family of 5
        - Kids play lots of online games
        - We all stream Netflix
        - Work from home sometimes
        - Large house, basement office
        - Can spend up to $150""",

        # Detailed requirements
        """Professional photographer here. I need the fastest possible internet for uploading large files
        to clients. I also do live streaming photography tutorials. Money isn't an issue, but reliability
        is crucial. I work from a downtown apartment.""",

        # Simple query
        "What's your best plan for a gamer who streams on Twitch?",

        # Question style
        """Can you help me pick a plan? I need good security because I do banking from home,
        and I want parental controls for my kids' devices. Speed doesn't need to be super fast,
        but I'd like something reliable."""
    ]

    print("Starting customer scenario testing...\n")

    for i, query in enumerate(sample_queries, 1):
        print(f"\n{'='*80}")
        print(f"Customer Query {i}:")
        print(query.strip())
        print("\nAnalyzing request...")

        full_prompt = base_prompt + "\n" + query

        try:
            response = system.analyze_scenario(full_prompt)
            print("\nRecommendations (JSON):")
            print(response)
        except Exception as e:
            print(f"Error processing query {i}: {str(e)}")
            # Add retry logic for temporary service issues
            if "503" in str(e):
                print("Service temporarily unavailable. Retrying in 5 seconds...")
                time.sleep(5)
                try:
                    response = system.analyze_scenario(full_prompt)
                    print("\nRecommendations (JSON) after retry:")
                    print(response)
                except Exception as retry_e:
                    print(f"Retry failed: {str(retry_e)}")

        print(f"{'='*80}")

        if i < len(sample_queries):
            print("\nProcessing next query in 2 seconds...")
            time.sleep(2)

def main():
    print("Testing ISP Recommendation System with Natural Queries...")
    test_customer_scenarios()
    print("\nTesting completed!")

if __name__ == "__main__":
    main()

Testing ISP Recommendation System with Natural Queries...


<ipython-input-2-7d790c3d2d70>:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Loading CSV data...
Processing 1000 customer records...


100%|██████████| 1000/1000 [00:06<00:00, 145.35it/s]


Created embeddings for 1000 customers
Starting customer scenario testing...


Customer Query 1:
Hi, I'm looking for internet for my new apartment. I live alone and mainly use it for Netflix 
        and some work from home. Don't need anything too fancy, trying to keep it under $50 if possible.

Analyzing request...

Recommendations (JSON):
```json
{
    "recommendations": {
        "internet_plan": {
            "name": "Fiber 500",
            "price": 45,
            "reasoning": "The customer's primary use cases are Netflix and work from home, which do not require extremely high speeds. Fiber 500 provides more than enough speed for these activities and is within their budget."
        },
        "recommended_addons": [
            {
                "name": "Wi-Fi Security",
                "price": 5,
                "reasoning": "As the customer works from home, it's essential to have a basic level of security to protect their personal and professional data. Wi-Fi Security provide